In [57]:
import spotipy
from spotipy import util
from spotipy.client import SpotifyException
from collections import Counter
from gensim.utils import tokenize
import time
import tqdm
import json

In [45]:
CLIENT_ID = '9d4442847caf4c278a9625e51438e8aa'
CLIENT_SECRET = '13b8ef1376cf4b7e98e356f43c1ad9cc'
USER_ID = 'DOsinga'

In [47]:
token = util.prompt_for_user_token(USER_ID, '',
                                   client_id=CLIENT_ID, client_secret=CLIENT_SECRET,
                                   redirect_uri='http://127.0.0.1:8000/callback')
session = spotipy.Spotify(auth=token)




            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Please navigate here: https://accounts.spotify.com/authorize?redirect_uri=http%3A%2F%2F127.0.0.1%3A8000%2Fcallback&response_type=code&client_id=9d4442847caf4c278a9625e51438e8aa


Enter the URL you were redirected to: http://127.0.0.1:8000/callback?code=AQAlzmWlglvRzGOl2khdQAzEWQmuCizwKOiEPOu5nqv3aDpRq67t4y3Kjz7VXY3vJeci6SvDEgs7MuVHbGvBKYcXzp3dy62W2fe0KHP2B1P8icZT5ZERtSLlbSt9LwBlQZP9BcfJHOSn0-2OEvZkmsUyT6mU8l72KFF7QkHMiccPSFt2a8XUJTmK6hED1a52SFW0RQ




In [48]:
def find_playlists(session, w, max_count=5000):
    try:
        res = session.search(w, limit=50, type='playlist')
        while res:
            for playlist in res['playlists']['items']:
                yield playlist
                max_count -= 1
                if max_count == 0:
                    raise StopIteration
            tries = 3
            while tries > 0:
                try:
                    res = session.next(res['playlists'])
                    tries = 0
                except SpotifyException as e:
                    tries -= 1
                    time.sleep(0.2)
                    if tries == 0:
                        raise
    except SpotifyException as e:
        status = e.http_status
        if status == 404:
            raise StopIteration
        raise

for pl in find_playlists(session, 'summer'):
    break

pl

{'collaborative': False,
 'external_urls': {'spotify': 'https://open.spotify.com/user/spotify/playlist/37i9dQZF1CAfsSPLpUyLzF'},
 'href': 'https://api.spotify.com/v1/users/spotify/playlists/37i9dQZF1CAfsSPLpUyLzF',
 'id': '37i9dQZF1CAfsSPLpUyLzF',
 'images': [{'height': 300,
   'url': 'https://i.scdn.co/image/5f13d953cbb925a7672811c3d9df13760c4cae60',
   'width': 300}],
 'name': 'Your Summer Rewind',
 'owner': {'display_name': 'Spotify',
  'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
  'href': 'https://api.spotify.com/v1/users/spotify',
  'id': 'spotify',
  'type': 'user',
  'uri': 'spotify:user:spotify'},
 'public': None,
 'snapshot_id': '83cvvTMD/RLiN6WXWsV2+QWDXuyx0q/rNNRib/DORlQWQOph3cdL+Q6Gz5rXCl/SNdaY6iHEEns=',
 'tracks': {'href': 'https://api.spotify.com/v1/users/spotify/playlists/37i9dQZF1CAfsSPLpUyLzF/tracks',
  'total': 30},
 'type': 'playlist',
 'uri': 'spotify:user:spotify:playlist:37i9dQZF1CAfsSPLpUyLzF'}

In [37]:
word_counts = Counter({'a': 1})
playlists = {}
words_seen = set()
playlists = {}
count = 0
dupes = 0

In [49]:
while len(playlists) < 100000:
    for word, _ in word_counts.most_common():
        if not word in words_seen:
            words_seen.add(word)
            print('word>', word)
            for playlist in find_playlists(session, word):
                if playlist['id'] in playlists:
                    dupes += 1
                elif playlist['name'] and playlist['owner']:
                    playlists[playlist['id']] = {
                      'owner': playlist['owner']['id'],
                      'name': playlist['name'],
                      'id': playlist['id'],
                    }
                    count += 1
                    for token in tokenize(playlist['name'], lowercase=True):
                        word_counts[token] += 1
            break

word> in
word> for
word> me
word> my
word> soundtrack
word> on
word> all
word> it
word> dance
word> mix


In [62]:
def track_yielder(session, playlist):
    res = session.user_playlist_tracks(playlist['owner'], playlist['id'],
              fields='items(track(id, name, artists(name, id), duration_ms)),next')
    while res:
        for track in res['items']:
            if track['track']:
                yield track['track']
        tries = 3
        while tries > 0:
            try:
                res = session.next(res)
                if not res or  not res.get('items'):
                    raise StopIteration
                tries = 0
            except SpotifyException as e:
                if 400 <= e.http_status <= 499:
                    raise StopIteration
                tries -= 1
                time.sleep(1)
                if tries == 0:
                    raise e


In [63]:
if os.path.isfile('data/songs.db'):
    os.remove('data/songs.db')
conn = sqlite3.connect('data/songs.db')
c = conn.cursor()
c.execute('CREATE TABLE songs (id text primary key, name text, artist text)')
c.execute('CREATE INDEX name_idx on songs(name)')

tracks_seen = set()
with open('data/playlists.ndjson', 'w') as fout_playlists:
    with open('data/songs_ids.txt', 'w') as fout_song_ids:
        for playlist in tqdm.tqdm(playlists.values()):
            fout_playlists.write(json.dumps(playlist) + '\n')
            track_ids = []
            for track in track_yielder(session, playlist):
                track_id = track['id']
                if not track_id:
                    continue
                if not track_id in tracks_seen:
                    c.execute("INSERT INTO songs VALUES (?, ?, ?)", 
                              (track['id'], track['name'], track['artists'][0]['name']))
                track_ids.append(track_id)
            fout_song_ids.write(' '.join(track_ids) + '\n')
            conn.commit()
conn.commit()


  0%|          | 2/102954 [00:00<8:12:34,  3.48it/s]
Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/douwe/proj/notebooks/venv3/lib/python3.5/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

  0%|          | 244/102954 [02:42<18:57:17,  1.51it/s]


KeyboardInterrupt: 